## Importing Dependencies

In [2]:
import requests
from htmldate import find_date
from datetime import datetime
import requests
import re
from bs4 import BeautifulSoup
import ssl
from tqdm import tqdm
from textblob import TextBlob
import openai

from html_extractor import *
from get_suburls import *
from openai_func import *

from keyword_extraction import keyword_extractor_paragraph as kep

## Extracting sub urls

In [3]:
urls_list = ["https://www.khaleejtimes.com"]
scraper = WebScraper(1) # KEEP IT 1, 2 or more will result in 1000's of urls.
                        # Integration with DB will make it faster in future, as fetching is much faster than scrapping.
inside_urls, failed_fetch, sub_url_size, total_size = scraper.get_suburls(urls_list)

print("Inside URLs:", inside_urls)
print("Failed Fetch:", failed_fetch)
print("Splits:", len(inside_urls))
print("Tree size:", total_size)

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00,  1.27it/s]

Inside URLs: {0: ['https://www.khaleejtimes.com'], 1: ['https://www.khaleejtimes.com/mykt', 'https://www.khaleejtimes.com/trading', 'https://www.khaleejtimes.com/contact-us', 'https://www.khaleejtimes.com/uae', 'https://www.khaleejtimes.com/uae/local-plus', 'https://www.khaleejtimes.com/weather', 'https://www.khaleejtimes.com/guide-to-uae', 'https://www.khaleejtimes.com/life-and-living/getting-a-loan', 'https://www.khaleejtimes.com/life-and-living/schools-in-the-uae', 'https://www.khaleejtimes.com/life-and-living/renting-in-the-uae', 'https://www.khaleejtimes.com/ramadan-2024', 'https://www.khaleejtimes.com/world', 'https://www.khaleejtimes.com/asia/india', 'https://www.khaleejtimes.com/asia/pakistan', 'https://www.khaleejtimes.com/world/americas', 'https://www.khaleejtimes.com/world/europe', 'https://www.khaleejtimes.com/world/africa', 'https://www.khaleejtimes.com/opinion', 'https://www.khaleejtimes.com/business', 'https://www.khaleejtimes.com/business/auto', 'https://www.khaleejtime

## Joining sub urls into one single list

In [4]:
website_urls = [item for sublist in list(inside_urls.values()) for item in sublist]
print(len(website_urls))

website_urls = website_urls[259:] # For testing Purposes
len(website_urls)

339


80

## Creating {url : html content} dictionary

In [ ]:
url_html_extracted = get_html(website_urls)
url_html_extracted

## Keyword extraction performed on above dictionary

In [ ]:
keywords = ["gaza"]
url_extracted_html = kep(website_content = url_html_extracted[0], keywords = keywords, filter_by_amount = 60)

url_extracted_html

## Converting dictionary to list of tuple pairs, for implementation of batches

In [ ]:
content_list = [(key,value[:1000]) for key, value in url_extracted_html.items()] # 1000 is temporary until tokenier function is not set up
content_list

## Cutting the above list fo batches of batch size MAX_CONTENT

In [10]:
MAX_CONTENT = 3

content_list_complete = []

iterations = len(content_list) // MAX_CONTENT

for i in range(iterations):
    sub_content_list = content_list[MAX_CONTENT * i: MAX_CONTENT * (i + 1)]
    content_list_complete.append(sub_content_list)

# Handle remaining elements after the loop
remaining_elements = content_list[MAX_CONTENT * iterations:]
if remaining_elements:
    iterations += 1
    content_list_complete.append(remaining_elements)

## Openai's api

In [11]:
question = "Whats going on in gaza?"

response_complete = ''
for data_idx in range(len(content_list_complete)):

    prompt = f""" 
        Data is in the form of tuples inside list: {content_list_complete[data_idx]} \n\n\n 
        Question: {question} \n\n\n
        Method of reply: 100 - 200 word sentences, clear reply,
        provide url if neccessary.
        """


    response = get_completion(prompt)
    response_complete += response + "\n\n"
    print(f"Batche {data_idx + 1} out of {iterations} completed ")
    

Batches 1 completed out of 4
Batches 2 completed out of 4
Batches 3 completed out of 4
Batches 4 completed out of 4


In [12]:
with open("Output.txt" , "w") as f:
    f.write(response_complete)